In [2]:
import numpy as np
from tifffile import imread, imsave
from joblib import Parallel, delayed
import os
import imageio

In [5]:
def process_image(seg_file, img_dir, output_dir):
    # Extract the base filename and replace 'seg__' with 'img__'
    base_filename = os.path.basename(seg_file).replace('seg__', 'img__')

    # Construct the full path to the corresponding img file
    img_file = os.path.join(img_dir, base_filename)

    # Read the segmentation and corresponding image
    seg = imread(seg_file)
    img = imread(img_file)

    # Create the mask: keep img where seg is non-zero, else 0
    mask = np.where(seg > 0, img, 0)

    # Save the mask image
    output_file = os.path.join(output_dir, base_filename.replace('img__', 'mask__'))
    imsave(output_file, mask.astype(np.uint8))

In [3]:
def process_image(seg_file, img_dir, output_dir):
    # Read the segmentation and corresponding image
    seg = imread(seg_file)
    img_file = os.path.join(img_dir, seg_file.split('/')[-1].replace('seg__', 'img__'))
    img = imread(img_file)

    # Create the mask: keep img where seg is non-zero, else 0
    mask = np.where(seg > 0, img, 0)

    # Save the mask image
    output_file = os.path.join(output_dir, seg_file.split('/')[-1].replace('seg__', 'mask__'))
    imsave(output_file, mask.astype(np.uint8))

In [6]:
def main():
    # Define directories
    root_dir = 'D:\shared_drive\oz'
    seg_dir = os.path.join(root_dir, 'seg')
    img_dir = os.path.join(root_dir, 'img')
    output_dir = os.path.join(root_dir, 'mask')

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get list of segmentation files
    seg_files = [os.path.join(seg_dir, f) for f in os.listdir(seg_dir) if f.endswith('.tif')]

    # Process each image in parallel
    Parallel(n_jobs=60)(delayed(process_image)(f, img_dir, output_dir) for f in seg_files)

if __name__ == "__main__":
    main()